In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
# https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install transformers
# !pip3 install kobert-transformers

In [55]:
# !pip install gluonnlp
# !pip install mxnet

In [8]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

device = torch.device("cuda:0")

In [9]:
import os
import ast
from tqdm import tqdm
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [10]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU가 사용 중입니다.")
else:
    device = torch.device("cpu")

GPU가 사용 중입니다.


In [11]:
path = '/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment'
rpath = os.path.join(path, 'maxEmotionTmp.csv')
tmp = pd.read_csv(rpath)
tmp['MaxEmotion'].value_counts()

기쁨        1227
일상적인       851
고마운        445
생각이 많은     138
설레는        134
슬픔          96
즐거운         63
짜증남         18
사랑하는        16
걱정스러운       11
힘듦           1
Name: MaxEmotion, dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for idx in range(len(tmp)):
  tmp['MaxEmotionLabel'] = le.fit_transform(tmp['MaxEmotion'])
tmp

,text,MaxScore,MaxEmotion,MaxEmotionLabel
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,0.628699,기쁨,2
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,0.741972,기쁨,2
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,0.511306,일상적인,7
3,아파트는 사진에서 보이는 것보다 더 넓습니다,0.793756,일상적인,7
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,0.696043,일상적인,7
...,...,...,...,...
2995,시간 지하철이 통과합니다,0.854543,일상적인,7
2996,윌리엄은 매우 친절했습니다,0.392459,일상적인,7
2997,온라인에 있는 침실 사진이 오래되었습니다,0.739786,일상적인,7
2998,훌륭한 위치 특히 햇살이 잘 드는 거실 공간과 발코니로 연결된 미닫이문이 마음에 들...,0.647639,기쁨,2


In [13]:
# rpath2 = os.path.join(path, 'sentimentTmp.csv')
# tmp2 = pd.read_csv(rpath2)
# tmp2.head(1)

In [14]:
data = tmp[['text', 'MaxEmotionLabel']]
data

,text,MaxEmotionLabel
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,2
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,2
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,7
3,아파트는 사진에서 보이는 것보다 더 넓습니다,7
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,7
...,...,...
2995,시간 지하철이 통과합니다,7
2996,윌리엄은 매우 친절했습니다,7
2997,온라인에 있는 침실 사진이 오래되었습니다,7
2998,훌륭한 위치 특히 햇살이 잘 드는 거실 공간과 발코니로 연결된 미닫이문이 마음에 들...,2


In [15]:
data_list = []
for ques, label in zip (data['text'], data['MaxEmotionLabel']):
  data = []
  data.append(ques)
  data.append(str(label))
  data_list.append(data)
data_list[:10]

[['더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다', '2'],
 ['게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼마켓과 일부 편의점 및 기차역이 정말 가깝습니다',
  '2'],
 ['처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다', '7'],
 ['아파트는 사진에서 보이는 것보다 더 넓습니다', '7'],
 ['커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다', '7'],
 ['서양인으로서 우리는 다다미 바닥에서 이상한 냄새가 난다는 것을 알았지만 나중에 모든 다다미 방에서 같은 냄새가 난다는 것을 알게 되었습니다 지하철역에서 도보로 약 분 거리에 위치 이 노선에서는 패스가 유효하지 않으며 슈퍼마켓 상점과 레스토랑에서 도보 거리에 있습니다',
  '4'],
 ['전반적으로 좋은 경험을 했으며 도쿄를 여행하는 모든 사람에게 추천합니다', '2'],
 ['아파트는 커플에게 적합합니다 조금 작지만 정말 완벽한 가구가 그것을 보완합니다 일상 생활에 필요한 모든 것이 제공됩니다', '7'],
 ['우리는 또한 동네를 많이 즐겼으며 주요 도시와 가까운 매력적인 장소입니다 머무르게 해주셔서 감사합니다', '1'],
 ['다시 방문하겠습니다', '1']]

In [16]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [17]:
len(dataset_train), len(dataset_test)

(2400, 600)

In [18]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [19]:
from kobert_tokenizer import KoBERTTokenizer
import torch
from transformers import BertModel
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize
# tokenizer.tokenize("설치 안되고 지랄이야")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [20]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #* epoch 조절해야함
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [21]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [22]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


In [23]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 11,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [24]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [25]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


In [26]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [37]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.4992032051086426 train acc 0.890625
epoch 1 train acc 0.8199013157894737


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 1 test acc 0.6817708333333333


  0%|          | 0/38 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6984375


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.35532665252685547 train acc 0.90625
epoch 5 train acc 0.8548519736842105


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 5 test acc 0.6619791666666667


In [41]:
torch.save(model, f'/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/에어비앤비2/huggingface_multiSentiment/SentimentAnalysisKOBert_StateDict.pt')

In [48]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [52]:
sentence = '데이터 적어서 과적합 난 듯' #* 일상적인
predict(sentence)

7

In [53]:
sentence = '설치 개빡치는데 되서 다행이야' #* 슬픔
predict(sentence)

6

In [50]:
le.classes_

array(['걱정스러운', '고마운', '기쁨', '사랑하는', '생각이 많은', '설레는', '슬픔', '일상적인', '즐거운',
       '짜증남', '힘듦'], dtype=object)